In [8]:
from typing import Tuple, List, Dict

import pandas as pd
import folium
import numpy as np
   

pd.set_option('display.precision', 2)


df_sites = pd.DataFrame(
    [['hotel',              48.8527, 2.3542],
     ['Sacre Coeur',        48.8867, 2.3431],
     ['Louvre',             48.8607, 2.3376],
     ['Montmartre',         48.8872, 2.3388],
     ['Port de Suffren',    48.8577, 2.2902],
     ['Arc de Triomphe',    48.8739, 2.2950],
     ['Av. Champs Élysées', 48.8710, 2.3036],
     ['Notre Dame',         48.8531, 2.3498],
     ['Tour Eiffel',        48.8585, 2.2945]],
    columns=pd.Index(['site', 'latitude', 'longitude'], name='paris')
)

df_sites

paris,site,latitude,longitude
0,hotel,48.85,2.35
1,Sacre Coeur,48.89,2.34
2,Louvre,48.86,2.34
3,Montmartre,48.89,2.34
4,Port de Suffren,48.86,2.29
5,Arc de Triomphe,48.87,2.29
6,Av. Champs Élysées,48.87,2.30
7,Notre Dame,48.85,2.35
8,Tour Eiffel,48.86,2.29


In [9]:
avg_location = df_sites[['latitude', 'longitude']].mean()
map_paris = folium.Map(location=avg_location, zoom_start=13)

for site in df_sites.itertuples():
    marker = folium.Marker(location=(site.latitude, site.longitude),
                           tooltip=site.site)
    marker.add_to(map_paris)

map_paris

df_route = df_sites.copy()
df_route.index.name = 'visit_order'

df_route

df_route_segments = df_route.join(
    df_route.shift(-1),  # map each stop to its next stop
    rsuffix='_next'
).dropna()  # last stop has no "next one", so drop it

df_route_segments

map_paris = folium.Map(location=avg_location, zoom_start=13)

for stop in df_route_segments.itertuples():
    # marker for current stop
    marker = folium.Marker(location=(stop.latitude, stop.longitude),
                           tooltip=stop.site)
    # line for the route segment connecting current to next stop
    line = folium.PolyLine(
        locations=[(stop.latitude, stop.longitude), 
                   (stop.latitude_next, stop.longitude_next)],
        tooltip=f"{stop.site} to {stop.site_next}",
    )
    # add elements to the map
    marker.add_to(map_paris)
    line.add_to(map_paris)

# maker for last stop wasn't added in for loop, so adding it now 
folium.Marker(location=(stop.latitude_next, stop.longitude_next),
              tooltip=stop.site_next).add_to(map_paris);

map_paris

map_paris = folium.Map(location=avg_location, zoom_start=13)

for stop in df_route_segments.itertuples():
    initial_stop = stop.Index == 0
    # icon for current stop
    icon = folium.Icon(icon='home' if initial_stop else 'info-sign', 
                       color='cadetblue' if initial_stop else 'red')
    # marker for current stop
    marker = folium.Marker(location=(stop.latitude, stop.longitude),
                           icon=icon, tooltip=stop.site)
    # line for the route segment connecting current to next stop
    line = folium.PolyLine(
        locations=[(stop.latitude, stop.longitude), 
                   (stop.latitude_next, stop.longitude_next)],
        tooltip=f"{stop.site} to {stop.site_next}",
    )
    # add elements to the map
    marker.add_to(map_paris)
    line.add_to(map_paris)

# When for loop ends, the stop variable has the second-to-last 
# stop in the route, so the marker for the last stop is missing 
# We add it now using the "next" columns of the last row
folium.Marker(
    location=(stop.latitude_next, stop.longitude_next),
    tooltip=stop.site_next, 
    icon = folium.Icon(icon='info-sign', color='red')
).add_to(map_paris);

from geoutils import GeoAnalyzer

from geopy.distance import geodesic

_Location = Tuple[float, float]


def ellipsoidal_distance(point1: _Location, point2: _Location) -> float:
    """Calculate ellipsoidal distance (in meters) between point1 and 
    point2 where each point is represented as a tuple (lat, lon)"""
    return geodesic(point1, point2).meters

df_route_segments['distance_seg'] = df_route_segments.apply(
    lambda stop: ellipsoidal_distance(
        (stop.latitude, stop.longitude), 
        (stop.latitude_next, stop.longitude_next)), 
    axis=1
)

df_route_segments

C:\Users\flymo\AppData\Roaming\Python\Python39\site-packages\folium\utilities.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])
C:\Users\flymo\AppData\Roaming\Python\Python39\site-packages\folium\utilities.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])


ModuleNotFoundError: No module named 'numpy.char'